Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
image_size = 28
num_labels = 10
beta = 0.001

graph = tf.Graph()

with graph.as_default():
  tf.set_random_seed(1234)
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.068167
Minibatch accuracy: 18.8%
Validation accuracy: 21.4%
Minibatch loss at step 500: 2.505962
Minibatch accuracy: 76.6%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.950267
Minibatch accuracy: 77.3%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 0.905230
Minibatch accuracy: 84.4%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 0.834005
Minibatch accuracy: 86.7%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 0.831445
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.753985
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Test accuracy: 89.0%


--Results before regularization  
Initialized  
Minibatch loss at step 0: 16.007515  
Minibatch accuracy: 18.8%  
Validation accuracy: 21.4%  
Minibatch loss at step 500: 1.201725  
Minibatch accuracy: 75.8%  
Validation accuracy: 75.3%  
Minibatch loss at step 1000: 1.745090  
Minibatch accuracy: 75.8%  
Validation accuracy: 76.2%  
Minibatch loss at step 1500: 0.583779  
Minibatch accuracy: 83.6%  
Validation accuracy: 77.0%  
Minibatch loss at step 2000: 0.909385  
Minibatch accuracy: 84.4%  
Validation accuracy: 77.1%  
Minibatch loss at step 2500: 0.976987  
Minibatch accuracy: 77.3%  
Validation accuracy: 78.0%  
Minibatch loss at step 3000: 0.922433  
Minibatch accuracy: 78.9%  
Validation accuracy: 78.6%  
Test accuracy: 85.9%  

--Results after regularization    
Initialized  
Minibatch loss at step 0: 19.068167  
Minibatch accuracy: 18.8%  
Validation accuracy: 21.4%  
Minibatch loss at step 500: 2.505962  
Minibatch accuracy: 76.6%  
Validation accuracy: 75.8%  
Minibatch loss at step 1000: 1.950267  
Minibatch accuracy: 77.3%  
Validation accuracy: 77.8%  
Minibatch loss at step 1500: 0.905230  
Minibatch accuracy: 84.4%  
Validation accuracy: 79.8%  
Minibatch loss at step 2000: 0.834005  
Minibatch accuracy: 86.7%  
Validation accuracy: 80.6%  
Minibatch loss at step 2500: 0.831445  
Minibatch accuracy: 81.2%  
Validation accuracy: 81.4%  
Minibatch loss at step 3000: 0.753985  
Minibatch accuracy: 82.8%  
Validation accuracy: 81.8%  
Test accuracy: 89.0%  

In [20]:
### Neural Network Version
batch_size = 128
image_size = 28
num_labels = 10
relu_layer_nodes = 1024
beta2 = 0.0
beta3 = 0.0

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(1234)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  y_ = tf.placeholder(tf.float32, shape=[None, 10])
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]), name='Weights')
  biases = tf.Variable(tf.zeros([num_labels]), name='biases')
  
  # Training computation.
  with tf.name_scope('logits_output_to_relu'):
###      logits_output_to_relu = tf.matmul(tf_train_dataset, weights) + biases
      logits_output_to_relu = tf_train_dataset
      print('tf_train_dataset shape = ', tf_train_dataset.get_shape())
      print('weights shape = ', weights.get_shape())
      print('biases shape = ', biases.get_shape())
      print('logits_output_to_relu shape = ', logits_output_to_relu.get_shape())
  
   # Variables.
  weights2 = tf.Variable(
    tf.truncated_normal([image_size*image_size, relu_layer_nodes]), name='weights2')
  biases2 = tf.Variable(tf.zeros([relu_layer_nodes]), name='biases2')
  print('weights2 shape = ', weights2.get_shape())
  print('biases2 shape = ', biases2.get_shape())
  input_to_relu = tf.matmul(logits_output_to_relu, weights2) + biases2
  print('input_to_relu shape = ', input_to_relu.get_shape())
  
  # Insert the ReLU
  with tf.name_scope('relu'):
      output_from_relu = tf.nn.relu(input_to_relu)
###      output_from_relu = input_to_relu
      print('output_from_relu shape = ', output_from_relu.get_shape())
    
    # Variables.
  weights3 = tf.Variable(
    tf.truncated_normal([relu_layer_nodes, num_labels]), name='weight3')
  biases3 = tf.Variable(tf.zeros([num_labels]), name='biases3')
  print('weights3 shape = ', weights3.get_shape())
  print('biases3 shape = ', biases3.get_shape())
    
  with tf.name_scope('output_layer'):  
      logits_final = tf.matmul(output_from_relu, weights3) + biases3
      print('logits_final shape = ', logits_final.get_shape())

  # Loss function
  with tf.name_scope('loss'):
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_final)) + \
        beta2*tf.nn.l2_loss(weights2) + beta3*tf.nn.l2_loss(weights3)  # what about biases?
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  with tf.name_scope('train_prediction'):
      train_prediction = tf.nn.softmax(logits_final)
      correct_prediction = tf.equal(tf.argmax(logits_final,1), tf.argmax(y_,1))
      test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
###  valid_prediction = tf.nn.softmax(
###    tf.matmul(tf_valid_dataset, weights) + biases)
###  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  train_writer = tf.summary.FileWriter('/tmp/tensorboard/run5',
                                      session.graph)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
###      print("Validation accuracy: %.1f%%" % accuracy(
###        valid_prediction.eval(), valid_labels))
###  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  feed_dict = {tf_train_dataset : test_dataset[:128,:], y_ : test_labels[:128]}
  print("test accuracy %g"%test_accuracy.eval(feed_dict=feed_dict))


tf_train_dataset shape =  (128, 784)
weights shape =  (784, 10)
biases shape =  (10,)
logits_output_to_relu shape =  (128, 784)
weights2 shape =  (784, 1024)
biases2 shape =  (1024,)
input_to_relu shape =  (128, 1024)
output_from_relu shape =  (128, 1024)
weights3 shape =  (1024, 10)
biases3 shape =  (10,)
logits_final shape =  (128, 10)
Initialized
Minibatch loss at step 0: 365.533630
Minibatch accuracy: 13.3%
Minibatch loss at step 500: 19.946838
Minibatch accuracy: 81.2%
Minibatch loss at step 1000: 13.830750
Minibatch accuracy: 77.3%
Minibatch loss at step 1500: 6.575778
Minibatch accuracy: 88.3%
Minibatch loss at step 2000: 3.732969
Minibatch accuracy: 85.9%
Minibatch loss at step 2500: 7.352654
Minibatch accuracy: 77.3%
Minibatch loss at step 3000: 3.163788
Minibatch accuracy: 82.0%
test accuracy 0.84375


Performance before regularization:
Initialized
Minibatch loss at step 0: 365.533630
Minibatch accuracy: 13.3%
Minibatch loss at step 500: 19.946838
Minibatch accuracy: 81.2%
Minibatch loss at step 1000: 13.830750
Minibatch accuracy: 77.3%
Minibatch loss at step 1500: 6.575778
Minibatch accuracy: 88.3%
Minibatch loss at step 2000: 3.732969
Minibatch accuracy: 85.9%
Minibatch loss at step 2500: 7.352654
Minibatch accuracy: 77.3%
Minibatch loss at step 3000: 3.163788
Minibatch accuracy: 82.0%
test accuracy 0.84375

Performance after regularization, beta2=0.001, beta3=0.001
Minibatch loss at step 0: 680.448730
Minibatch accuracy: 13.3%
Minibatch loss at step 500: 201.892426
Minibatch accuracy: 82.8%
Minibatch loss at step 1000: 115.419212
Minibatch accuracy: 78.9%
Minibatch loss at step 1500: 68.644249
Minibatch accuracy: 90.6%
Minibatch loss at step 2000: 41.235996
Minibatch accuracy: 89.8%
Minibatch loss at step 2500: 25.212269
Minibatch accuracy: 86.7%
Minibatch loss at step 3000: 15.469172
Minibatch accuracy: 89.1%
test accuracy 0.921875

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [13]:
### Neural Network Version - Overfitting to training data on purpose.
batch_size = 128
image_size = 28
num_labels = 10
relu_layer_nodes = 1024
beta2 = 0.0
beta3 = 0.0

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(1234)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  y_ = tf.placeholder(tf.float32, shape=[None, 10])
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]), name='Weights')
  biases = tf.Variable(tf.zeros([num_labels]), name='biases')
  
  # Training computation.
  with tf.name_scope('logits_output_to_relu'):
###      logits_output_to_relu = tf.matmul(tf_train_dataset, weights) + biases
      logits_output_to_relu = tf_train_dataset
      print('tf_train_dataset shape = ', tf_train_dataset.get_shape())
      print('weights shape = ', weights.get_shape())
      print('biases shape = ', biases.get_shape())
      print('logits_output_to_relu shape = ', logits_output_to_relu.get_shape())
  
   # Variables.
  weights2 = tf.Variable(
    tf.truncated_normal([image_size*image_size, relu_layer_nodes]), name='weights2')
  biases2 = tf.Variable(tf.zeros([relu_layer_nodes]), name='biases2')
  print('weights2 shape = ', weights2.get_shape())
  print('biases2 shape = ', biases2.get_shape())
  input_to_relu = tf.matmul(logits_output_to_relu, weights2) + biases2
  print('input_to_relu shape = ', input_to_relu.get_shape())
  
  # Insert the ReLU
  with tf.name_scope('relu'):
      output_from_relu = tf.nn.relu(input_to_relu)
###      output_from_relu = input_to_relu
      print('output_from_relu shape = ', output_from_relu.get_shape())
    
    # Variables.
  weights3 = tf.Variable(
    tf.truncated_normal([relu_layer_nodes, num_labels]), name='weight3')
  biases3 = tf.Variable(tf.zeros([num_labels]), name='biases3')
  print('weights3 shape = ', weights3.get_shape())
  print('biases3 shape = ', biases3.get_shape())
    
  with tf.name_scope('output_layer'):  
      logits_final = tf.matmul(output_from_relu, weights3) + biases3
      print('logits_final shape = ', logits_final.get_shape())

  # Loss function
  with tf.name_scope('loss'):
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_final)) + \
        beta2*tf.nn.l2_loss(weights2) + beta3*tf.nn.l2_loss(weights3)  # what about biases?
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  with tf.name_scope('train_prediction'):
      train_prediction = tf.nn.softmax(logits_final)
      correct_prediction = tf.equal(tf.argmax(logits_final,1), tf.argmax(y_,1))
      test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
###  valid_prediction = tf.nn.softmax(
###    tf.matmul(tf_valid_dataset, weights) + biases)
###  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  train_writer = tf.summary.FileWriter('/tmp/tensorboard/run5',
                                      session.graph)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
###    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
###      print("Validation accuracy: %.1f%%" % accuracy(
###        valid_prediction.eval(), valid_labels))
###  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  feed_dict = {tf_train_dataset : test_dataset[:128,:], y_ : test_labels[:128]}
  print("test accuracy %g"%test_accuracy.eval(feed_dict=feed_dict))

tf_train_dataset shape =  (128, 784)
weights shape =  (784, 10)
biases shape =  (10,)
logits_output_to_relu shape =  (128, 784)
weights2 shape =  (784, 1024)
biases2 shape =  (1024,)
input_to_relu shape =  (128, 1024)
output_from_relu shape =  (128, 1024)
weights3 shape =  (1024, 10)
biases3 shape =  (10,)
logits_final shape =  (128, 10)
Initialized
Minibatch loss at step 0: 365.533630
Minibatch accuracy: 13.3%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
test accuracy 0.726562


Initialized  
Minibatch loss at step 0: 365.533630  
Minibatch accuracy: 13.3%  
Minibatch loss at step 500: 0.000000  
Minibatch accuracy: 100.0%  
Minibatch loss at step 1000: 0.000000  
Minibatch accuracy: 100.0%  
Minibatch loss at step 1500: 0.000000  
Minibatch accuracy: 100.0%  
Minibatch loss at step 2000: 0.000000  
Minibatch accuracy: 100.0%  
Minibatch loss at step 2500: 0.000000  
Minibatch accuracy: 100.0%  
Minibatch loss at step 3000: 0.000000  
Minibatch accuracy: 100.0%  
test accuracy 0.726562  

Discussion:  As can be seen above, the minibatch is exactly classified (with zero loss correspondingly).  However, the test accuracy is considerably lower (0.72 vs. 0.84 - the non-regularized case).

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [60]:
### Neural Network Version - Adding dropouts
batch_size = 128
image_size = 28
num_labels = 10
relu_layer_nodes = 1024
beta2 = 0.0
beta3 = 0.0

tf.reset_default_graph()
###graph = tf.Graph()
session = tf.Session()
with session.as_default():
  tf.set_random_seed(1234)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  y_ = tf.placeholder(tf.float32, shape=[None, 10])
  keep_prob_val = tf.placeholder(tf.float32,shape=())
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels],seed=12345), name='Weights')
  biases = tf.Variable(tf.zeros([num_labels]), name='biases')
  
  # Training computation.
  with tf.name_scope('logits_output_to_relu'):
###      logits_output_to_relu = tf.matmul(tf_train_dataset, weights) + biases
      logits_output_to_relu = tf_train_dataset
      print('tf_train_dataset shape = ', tf_train_dataset.get_shape())
      print('weights shape = ', weights.get_shape())
      print('biases shape = ', biases.get_shape())
      print('logits_output_to_relu shape = ', logits_output_to_relu.get_shape())
  
   # Variables.
  weights2 = tf.Variable(
    tf.truncated_normal([image_size*image_size, relu_layer_nodes],seed=12347), name='weights2')
  biases2 = tf.Variable(tf.zeros([relu_layer_nodes]), name='biases2')
  print('weights2 shape = ', weights2.get_shape())
  print('biases2 shape = ', biases2.get_shape())
  input_to_relu = tf.matmul(logits_output_to_relu, weights2) + biases2
  print('input_to_relu shape = ', input_to_relu.get_shape())
  
  # Insert the ReLU
  with tf.name_scope('relu'):
      output_from_relu_pd = tf.nn.relu(input_to_relu)
      output_from_relu = tf.nn.dropout(output_from_relu_pd, keep_prob_val, seed=123477)
###      output_from_relu = output_from_relu_pd
      print('output_from_relu shape = ', output_from_relu.get_shape())
    
    # Variables.
  weights3 = tf.Variable(
    tf.truncated_normal([relu_layer_nodes, num_labels],seed=12349), name='weight3')
  biases3 = tf.Variable(tf.zeros([num_labels]), name='biases3')
  print('weights3 shape = ', weights3.get_shape())
  print('biases3 shape = ', biases3.get_shape())
    
  with tf.name_scope('output_layer'):  
      logits_final = tf.matmul(output_from_relu, weights3) + biases3
      print('logits_final shape = ', logits_final.get_shape())

  # Loss function
  with tf.name_scope('loss'):
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_final)) + \
        beta2*tf.nn.l2_loss(weights2) + beta3*tf.nn.l2_loss(weights3)  # what about biases?
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  with tf.name_scope('train_prediction'):
      train_prediction = tf.nn.softmax(logits_final)
      correct_prediction = tf.equal(tf.argmax(logits_final,1), tf.argmax(y_,1))
      test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
###  valid_prediction = tf.nn.softmax(
###    tf.matmul(tf_valid_dataset, weights) + biases)
###  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

  num_steps = 3001
###with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")

#
# Some diagnostics
#
###  print('output_from_relu_bd = ', output_from_relu_pd.eval())
###  print('keep_prod_array = ', keep_prod_array.eval())
  

  train_writer = tf.summary.FileWriter('/tmp/tensorboard/run5',
                                      session.graph)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob_val : 0.75}
    
    _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
###      print("Validation accuracy: %.1f%%" % accuracy(
###        valid_prediction.eval(), valid_labels))
###  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  print ('weights3 = ', session.run(weights3[1:10,1], feed_dict=feed_dict ))
###  print ('keep_prod_array = ', session.run(keep_prod_array[1:10,1], feed_dict=feed_dict ))  
  feed_dict = {tf_train_dataset : test_dataset[:128,:], y_ : test_labels[:128], keep_prob_val : 1.0}
  print("test accuracy %g"%test_accuracy.eval(feed_dict=feed_dict))

tf_train_dataset shape =  (128, 784)
weights shape =  (784, 10)
biases shape =  (10,)
logits_output_to_relu shape =  (128, 784)
weights2 shape =  (784, 1024)
biases2 shape =  (1024,)
input_to_relu shape =  (128, 1024)
output_from_relu shape =  (128, 1024)
weights3 shape =  (1024, 10)
biases3 shape =  (10,)
logits_final shape =  (128, 10)
Initialized
Minibatch loss at step 0: 412.749756
Minibatch accuracy: 6.2%
Minibatch loss at step 500: 28.077713
Minibatch accuracy: 79.7%
Minibatch loss at step 1000: 10.874871
Minibatch accuracy: 78.1%
Minibatch loss at step 1500: 9.256192
Minibatch accuracy: 84.4%
Minibatch loss at step 2000: 3.333242
Minibatch accuracy: 84.4%
Minibatch loss at step 2500: 3.821290
Minibatch accuracy: 82.0%
Minibatch loss at step 3000: 1.878068
Minibatch accuracy: 77.3%
weights3 =  [-0.22689311 -0.18822454 -0.28015491  0.32796231 -0.04363746 -0.0855078
 -0.05764526 -0.11022235  0.13837141]
test accuracy 0.867188


Minibatch loss at step 3000: 3.163788  
Minibatch accuracy: 82.0%  
test accuracy 0.84375

With keep_prob_array defined, but operation level seed added:
Minibatch accuracy: 81.2%
weights3 =  [-0.94319594 -0.55473816 -0.15087561 -0.02617398 -0.2992973  -0.30472267
 -0.0541427  -0.29530555  0.47049353]
test accuracy 0.867188

Keep_prob = 0.5:  
Minibatch accuracy: 72.7%  
weights3 =  [-0.36362457  0.18068232  0.37608263  0.1037811  -0.17209613 -0.18757257
 -0.34064397  0.49238163  0.39803791]  
test accuracy 0.898438  

Keep_prob = 0.75:
weights3 =  [-0.22689311 -0.18822454 -0.28015491  0.32796231 -0.04363746 -0.0855078
 -0.05764526 -0.11022235  0.13837141]
test accuracy 0.867188

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
